In [1]:
import os

from tqdm import tqdm

import pandas as pd
import numpy as np

from run_kmeans import RunKmeans

from os.path import join, splitext
from numpy import min, mean, sum

In [2]:
pwd = os.getcwd()
datadir = join(pwd, 'data/')
outdir = join(pwd, 'out/')
files = os.listdir(datadir)
filename = 'cloud.csv'
file = join(datadir, filename)
num_clusters = 25
outdir = join(outdir, str(num_clusters))
inits = [
    'random',
    'k-means++',
    'coc',
    'orss',
]

In [3]:
dfs = []
for file in tqdm(files[:]):
    filepath = join(datadir, file)
    comparator = RunKmeans(
        n_clusters=10,
        num_exp=20,
        file=filepath,
        do_pca=True,
    )
    inertias, times, iters = comparator.run_kmeans()
#     print(file)
    rows = []
    for k in inertias:
        for inertia in inertias[k]:
            if type(inertia) == type(np.float64(1.0)):
                continue
            elif type(inertia) == type(np.int64(1.0)):
                continue
            else:
                raise TypeError(inertia, 'not float or int')
        row = []
        row.append(k)
#         print(k, mean(inertias[k]), mean(times[k]))
        row.append(round(mean(inertias[k]), 2))
        row.append(round(min(inertias[k]), 2))
        row.append(round(mean(times[k]), 2))
        row.append(round(min(times[k]), 2))
        row.append(round(mean(iters[k]), 2))
        row.append(round(min(iters[k]), 2))
        rows.append(row)
        columns=[
            'Initialization', 
            'Average Inertia', 
            'Minimum Inertia',
            'Average Time',
            'Minimum Time',
            'Average Iteration',
            'Minimum Iteration',
        ]
    df = pd.DataFrame(rows, columns=columns)
    dfs.append(df)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [08:56<00:00, 59.66s/it]


In [4]:
for i, df in enumerate(dfs):
    print(files[i])
    print(df.head())

airlines.csv
  Initialization  Average Inertia  Minimum Inertia  Average Time  \
0         random              NaN              NaN          7.01   
1      k-means++     2.683300e+12     2.621426e+12          4.54   
2           orss              NaN              NaN          3.67   
3            coc              NaN              NaN          5.76   

   Minimum Time  Average Iteration  Minimum Iteration  
0          5.51                NaN                NaN  
1          2.33               45.0               19.0  
2          2.01                NaN                NaN  
3          3.62                NaN                NaN  
boston.csv
  Initialization  Average Inertia  Minimum Inertia  Average Time  \
0         random       1179535.38        726875.22          0.18   
1      k-means++        796562.49        707943.36          0.17   
2           orss        783434.35        708148.86          0.19   
3            coc        836074.29        714810.01          0.22   

   Minimum Tim

In [5]:
for i, df in enumerate(dfs):
    name, ext = splitext(files[i])
    file = join(outdir, name+'_out.csv')
    df.to_csv(file, index=False)